In [1]:
!pip install -q pandas tqdm 
!pip install -q transformers==4.55.0 # llm requires >=4.46.0
!pip install -q safetensors==0.4.3 # downgrade for torch 2.1.0
!pip install -q bitsandbytes==0.43.2 accelerate==1.9.0 # quantization
!pip install -q peft==0.17.0 trl==0.21.0 # finetune
!pip install -q datasets wandb


[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import os, pandas as pd, json, random
from datasets import Dataset, concatenate_datasets
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from trl import SFTTrainer, SFTConfig
import torch
import re
import wandb
import datetime

In [3]:
!wandb login 80f8e961aab3ee77e8dc19ddc1bfd4604203e400

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


## 데이터 로드

In [ ]:
# 객관식 여부 판단 함수
def is_multiple_choice(question_text):
    question_text = question_text.replace("\\n", "\n")
    lines = question_text.strip().split("\n")
    pat = re.compile(r"^\s*[\(\[]?\d{1}[\)\]\.]?\s+")
    option_count = sum(bool(pat.match(line)) for line in lines)
    return option_count >= 2

# 질문과 선택지 분리 함수
OPTION_LINE = re.compile(r"^\s*(?:[\(\[]?\d{1,2}[\)\]\.]?|[\u2460-\u2473])\s+")
def extract_question_and_choices(full_text: str):
    """
    전체 질문 문자열에서 질문 본문과 선택지 리스트를 분리
    - "\\n" 같은 리터럴 줄바꿈도 처리
    - 옵션이 여러 줄로 이어지면 뒤줄을 같은 선택지에 이어 붙임
    - 옵션 앞 번호/기호는 유지
    """
    # 리터럴 줄바꿈과 다양한 개행 정규화
    text = (full_text
            .replace("\\r\\n", "\n")
            .replace("\\n", "\n")
            .replace("\r\n", "\n")
            .replace("\r", "\n"))

    lines = [ln.strip() for ln in text.strip().split("\n") if ln.strip()]

    q_lines = []
    options = []
    in_options = False

    for ln in lines:
        if OPTION_LINE.match(ln):
            in_options = True
            options.append(ln.strip())  # 번호 포함 그대로
        else:
            if in_options and options:
                # 옵션 줄 다음 줄이 붙는 경우
                options[-1] += " " + ln
            else:
                q_lines.append(ln)

    question = " ".join(q_lines).strip()
    return question, options


# 프롬프트 생성기
def make_prompt_auto(row):
    Question = str(row["Question"]).strip()
    Answer = str(row["Answer"]).split("답변:")[-1].strip()
    if is_multiple_choice(Question):
        question, options = extract_question_and_choices(Question)
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                # "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                "아래 질문에 대해 적절한 선택지를 출력하세요.\n\n"
                f"질문: {question}\n"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
                )
    else:
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                "아래 주관식 질문에 대해 정확하고 간략한 설명을 작성하세요.\n\n"
                f"질문: {Question}\n\n"
                "답변:"
                )
    response = Answer

    return prompt, response

In [5]:
# 데이터 로드

dfs = []
dfs.append(pd.read_csv("../data/CyberMetric/mcqa_enhanced.csv"))
dfs.append(pd.read_csv("../data/FinShibainu/mcqa_enhanced.csv"))
dfs.append(pd.read_csv("../data/FinShibainu/qa.csv"))
dfs.append(pd.read_csv("../data/SecBench/mcqa_enhanced.csv"))
dfs.append(pd.read_csv("../data/SecBench/qa.csv"))

full = pd.concat(dfs, ignore_index=True)

In [6]:
full

,Question,Answer
0,다음 중 정보의 비밀을 나타내는 것은 무엇입니까?\n1. 가용성\n2. 인증\n3....,"답변: 4, 기밀성"
1,"어떤 유형의 인증은 당신이 아는 것, 당신이 가진 것, 당신이있는 것과 같은 여러 ...","답변: 3, 다중 인증 인증"
2,발가락은 무엇을 의미합니까?\n1. 평가 목표\n2. 평가 시간\n3. 평가 유형\...,"답변: 1, 평가 목표"
3,시스템이 리소스에 대한 액세스를 요청하는 사용자가 실제로 자신이 주장하는 사람인지 ...,"답변: 4, 인증"
4,"기밀성, 무결성 및 데이터 및 자산의 가용성에 대한 확인 및 보증을 포함하여 정보 ...","답변: 2, 정보 보증"
...,...,...
100508,블록 체인 기술에서 거래 검증 가능성을 유지하면서 거래 개인 정보를 향상시키기 위해...,답변: 거래 개인 정보 보호를 향상시키기 위해 제로 지식 증거를 사용하는 방법은 당...
100509,Linux 시스템에서 실제 권한 에스컬레이션 취약성 (예 : CVE-2019-186...,답변: CVE-2019-18634는 Linux 시스템에 영향을 미치는 권한 에스컬레...
100510,PHP에서 문자열을 정의하기 위해 이중 인용문과 단일 따옴표 사용의 차이점은 무엇입니까?,답변: PHP에서 이중 인용문으로 정의 된 문자열은 변수를 구문 분석하는 반면 단일...
100511,PHP 코드를 식별하기 위해 PHP에 구문 분석 태그를 작성하는 방법은 무엇입니까?,"답변: PHP에서 구문 분석 태그를 작성하는 세 가지 방법, 즉 '<? php?>'..."


In [ ]:
# 튜플을 딕셔너리로 변환
tuples = [make_prompt_auto(r) for _, r in full.iterrows()]
records = [{"prompt": prompt, "completion": response} for prompt, response in tuples] # SFTTrainer 에서 completion 필드 사용
random.seed(42)
random.shuffle(records)

# 10% 샘플링 (테스트용)
sample_size = int(len(records) * 0.1)
sampled_records = records[:sample_size]
print(f"전체 데이터: {len(records):,}개 → 샘플링: {len(sampled_records):,}개 (10%)")

# 간단 split
# n = int(len(records)*0.95)
n = int(len(sampled_records)*0.95)
train_ds = Dataset.from_list(sampled_records[:n])
eval_ds  = Dataset.from_list(sampled_records[n:])

전체 데이터: 100,513개 → 샘플링: 10,051개 (10%)


In [8]:
print(train_ds, eval_ds)
print(train_ds[0]) # 짧은 MCQA
print(train_ds[1]) # 긴 QA

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 9548
}) Dataset({
    features: ['prompt', 'completion'],
    num_rows: 503
})
{'prompt': '당신은 금융보안 전문가입니다.\n아래 질문에 대해 적절한 선택지를 출력하세요.\n\n질문: 공공민간협력(PPP) 사업에서 정부의 역할로 적절한 것은 무엇인가?\n선택지:\n1. 자금 부족 시 민간의 복지 관련 책임까지 담당한다.\n2. 프로젝트의 모든 결정 사항을 단독으로 처리한다.\n3. 민간의 이익을 우선하며 정부 역할을 최소화한다.\n4. 공적자금을 투입하고 민간과 함께 위험을 분담한다.\n\n답변:', 'completion': '4, 공적자금을 투입하고 민간과 함께 위험을 분담한다.'}
{'prompt': '당신은 금융보안 전문가입니다.\n아래 주관식 질문에 대해 정확하고 간략한 설명을 작성하세요.\n\n질문: CLI 감소 시 기업의 의사결정에는 어떤 영향을 미칠까요?\n\n답변:', 'completion': 'CLI(Consumer Leading Indicator, 소비자 선행 지표)가 감소할 경우, 이는 기업의 의사결정에 다양한 방식으로 영향을 미칠 수 있습니다. CLI는 일반적으로 소비자 신뢰도와 소비 지출 패턴을 반영하는 지표로 여겨지며, 감소는 소비자들이 경제에 대한 불확실성을 느끼거나 지출을 줄일 가능성을 암시합니다. 다음은 CLI 감소가 기업의 의사결정에 미치는 주요 영향입니다:\n\n### 1. **판매 예측 조정**  \nCLI가 감소하면 기업은 소비자 수요 감소를 예상하게 됩니다. 이는 다음과 같은 의사결정으로 이어집니다:  \n- **재고 관리:** 기업은 판매 감소에 대비해 재고를 줄이거나 보충 계획을 수정할 수 있습니다.  \n- **생산 계획:** 생산을 축소하거나 휴업하는 결정을 내릴 수 있습니다.  \n\n### 2. **마케팅 전략 수정**  \n소비

## QLoRA 세팅

In [ ]:
# 모델 선택
models = [
    "gemma-ko-7b", # baseline
    "ax-4.0-light-7b", # skt
    # "polyglot-12.8b",
    # "koalpaca-polyglot-12.8b",
    "midm-2.0-11.5b", # kt
    # "HyperCLOVAX-SEED-Think-14B", # naver
    # "kanana-1.5-15.7b-a3b-instruct", # kakao
    # "exaone-4.0-32b" # lg
]
selected_model = models[1]
model_path = f"../../models/{selected_model}" # 로컬 저장 모델 경로

# 4bit 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16, # NaN 방지
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# # 8bit 설정
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,          # 4bit → 8bit
#     llm_int8_threshold=6.0,     # 기본값 (필요 시 조정)
#     llm_int8_has_fp16_weight=False  # True로 하면 일부 레이어 FP16 유지
# )

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    # padding_side="left",
    padding_side="right" # 학습 시 right 권장
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

# 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto", # GPU 자동 배정
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    attn_implementation="eager",
)
base_model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
base_model # 모델 확인. target_modules에 입력

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(102400, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-05)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-05)
      )
    )
    (norm): Qwen2RMSNorm((3584,), 

In [ ]:
# QLoRA Configs

base_model = prepare_model_for_kbit_training(base_model)

# LoRA (Low-Rank Adaptation) 설정
peft_config = LoraConfig(
    # LoRA 핵심 파라미터
    r=16,                       # 랭크(rank): LoRA 행렬의 차원, 낮을수록 파라미터 적음, 높을수록 표현력 증가
                                # 일반적으로 8~64 사용, 16은 성능과 효율성의 균형점
    
    lora_alpha=32,              # 스케일링 파라미터: LoRA 업데이트의 강도 조절
                                # 일반적으로 r의 2배 설정 (16*2=32)
                                # 높을수록 LoRA의 영향력 증가
    
    lora_dropout=0.05,          # 드롭아웃 비율: 과적합 방지
                                # 0.05 = 5% 드롭아웃, 일반적으로 0.05~0.1 사용
    
    # 적용할 모듈 지정 (Transformer의 주요 선형 레이어들)
    target_modules=[
        "q_proj",               # Query 프로젝션 레이어
        "k_proj",               # Key 프로젝션 레이어  
        "v_proj",               # Value 프로젝션 레이어
        "o_proj",               # Output 프로젝션 레이어
        "gate_proj",            # Gate 프로젝션 레이어 (FFN)
        "up_proj",              # Up 프로젝션 레이어 (FFN)
        "down_proj"             # Down 프로젝션 레이어 (FFN)
    ],
    
    task_type="CAUSAL_LM"       # 태스크 타입: 인과적 언어모델 (다음 토큰 예측)
)

In [ ]:
# 개선된 훈련 설정
tokenizer.model_max_length = 128 # 긴 답변 필요 없음

cfg = SFTConfig(
    output_dir=f"../../models/{selected_model}_qlora",
    
    # 훈련 스케줄링 - 본격 학습용 설정
    num_train_epochs=2,            # 2-10 수준 테스트 필요
    per_device_train_batch_size=8, # 실제 학습 시 확대 (GPU VRAM 체크)
    gradient_accumulation_steps=8, # optimizer step 주기
    
    # 학습률 및 최적화
    learning_rate=1e-4,           # 안정적인 학습률
    weight_decay=0.01,            # 과적합 방지
    max_grad_norm=1.0,            # 그래디언트 클리핑
    
    # 로깅 및 저장 (개선된 주기)
    logging_steps=16,             
    logging_strategy="steps",
    # logging_first_step=True,
    save_steps=64,               
    # save_total_limit=3,         # 최대 3개 체크포인트 유지
    
    # 평가 설정
    eval_strategy="steps",
    eval_steps=64,               
    load_best_model_at_end=True,  # 최고 성능 모델 자동 선택
    metric_for_best_model="eval_loss",
    
    # 하드웨어 최적화
    bf16=True,
    dataloader_num_workers=4,
    dataloader_pin_memory=True,
    
    # 학습률 스케줄러
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    
    # 메모리 최적화
    gradient_checkpointing=True,
    
    # 기타 설정
    remove_unused_columns=False,
    report_to=["wandb"],            
    run_name=f"{selected_model}_qlora_{datetime.datetime.now().strftime('%m%d_%H%M')}",
    # disable_tqdm=True,            # tqdm이 stdout 삼키는 이슈 회피
)

print("✅ 훈련 설정 완료!")
print(f"총 에포크: {cfg.num_train_epochs}")
print(f"실제 배치 크기: {cfg.per_device_train_batch_size} × {cfg.gradient_accumulation_steps} = {cfg.per_device_train_batch_size * cfg.gradient_accumulation_steps}")
print(f"로깅 주기: {cfg.logging_steps}스텝마다")


✅ 훈련 설정 완료!
총 에포크: 2
실제 배치 크기: 8 × 8 = 64
로깅 주기: 16스텝마다


## 데이터 전처리

In [14]:
def truncate_keep_all_samples(dataset, max_length=None):
    """모든 샘플을 보존하면서 길이만 조정"""
    if max_length is None:
        max_length = tokenizer.model_max_length
    
    def truncate_sample(sample):
        prompt = sample["prompt"]
        completion = sample["completion"]
        
        # 전체 길이 체크
        full_text = prompt + completion
        full_tokens = tokenizer.encode(full_text, add_special_tokens=True)
        
        if len(full_tokens) <= max_length:
            return sample  # 그대로 유지
        
        # 프롬프트가 너무 길면 프롬프트도 자르기
        prompt_tokens = tokenizer.encode(prompt, add_special_tokens=False)
        completion_tokens = tokenizer.encode(completion, add_special_tokens=False)
        
        total_needed = len(prompt_tokens) + len(completion_tokens) + 2  # BOS/EOS
        
        if total_needed <= max_length:
            return sample  # 실제로는 문제없음
        
        # 프롬프트:완성 = 7:3 비율로 할당
        prompt_quota = int(max_length * 0.7)
        completion_quota = max_length - prompt_quota - 2
        
        # 프롬프트 자르기 (뒤쪽 유지)
        if len(prompt_tokens) > prompt_quota:
            truncated_prompt_tokens = prompt_tokens[-prompt_quota:]
            truncated_prompt = tokenizer.decode(truncated_prompt_tokens, skip_special_tokens=True)
        else:
            truncated_prompt = prompt
            completion_quota = max_length - len(prompt_tokens) - 2
        
        # 완성 자르기 (앞쪽 유지)
        if len(completion_tokens) > completion_quota:
            truncated_completion_tokens = completion_tokens[:completion_quota]
            truncated_completion = tokenizer.decode(truncated_completion_tokens, skip_special_tokens=True)
        else:
            truncated_completion = completion
        
        # # EOS 토큰 보장
        # if not truncated_completion.endswith('<|im_end|>'):
        #     truncated_completion += '<|im_end|>'
        
        return {"prompt": truncated_prompt, "completion": truncated_completion}
    
    return dataset.map(truncate_sample, num_proc=1)


# 모든 샘플 보존하며 자르기
final_train_ds = truncate_keep_all_samples(train_ds)
final_eval_ds = truncate_keep_all_samples(eval_ds)

print(f"✅ 원본 훈련 샘플: {len(train_ds):,}개")
print(f"✅ 최종 훈련 샘플: {len(final_train_ds):,}개")
print(f"✅ 제거된 샘플: 0개 (모든 샘플 보존!)")

# 길이 검증
max_lens = []
for i in range(min(100, len(final_train_ds))):
    sample = final_train_ds[i]
    tokens = tokenizer.encode(sample["prompt"] + sample["completion"], add_special_tokens=True)
    max_lens.append(len(tokens))

print(f"✅ 최대 토큰 수: {max(max_lens)}")
print(f"✅ 평균 토큰 수: {sum(max_lens)/len(max_lens):.1f}")

# 변수 업데이트
train_ds = final_train_ds
eval_ds = final_eval_ds

Map:   0%|          | 0/9548 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (558 > 128). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/503 [00:00<?, ? examples/s]

✅ 원본 훈련 샘플: 9,548개
✅ 최종 훈련 샘플: 9,548개
✅ 제거된 샘플: 0개 (모든 샘플 보존!)
✅ 최대 토큰 수: 127
✅ 평균 토큰 수: 113.5


In [15]:
# tokenize 오류 해결

import unicodedata, re

ZWS = "\u200b"          # zero-width space
NBSP = "\xa0"           # non-breaking space

def clean_text(s: str) -> str:
    s = s.replace("\r\n","\n").replace("\r","\n")   # 개행 통일
    s = s.replace(NBSP," ").replace(ZWS,"")         # 이상 공백 제거
    s = unicodedata.normalize("NFKC", s)            # 유니코드 정규화
    # 문자열에 직접 박아둔 특수토큰은 제거(토크나이저에 맡김)
    for tok in ("<s>","</s>","<bos>","</eos>","<<SYS>>","<<USER>>","<<ASSISTANT>>", "<|im_end|>"):
        s = s.replace(tok,"")
    return s

def normalize_pair(ex):
    p = clean_text(ex["prompt"]).rstrip()           # 뒤 공백/개행 제거
    c = clean_text(ex["completion"]).lstrip()       # 앞 공백 제거
    if not p.endswith("\n"):                        # 경계 개행 1개 강제
        p += "\n"
    return {"prompt": p, "completion": c}

train_ds = train_ds.map(normalize_pair, num_proc=4)
eval_ds  = eval_ds.map(normalize_pair,  num_proc=4)

Map (num_proc=4):   0%|          | 0/9548 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/503 [00:00<?, ? examples/s]

In [16]:
def has_non_empty_completion(ex):
    return (ex["completion"] or "").strip() != ""

train_ds = train_ds.filter(has_non_empty_completion)
eval_ds  = eval_ds.filter(has_non_empty_completion)
print("[filter] after drop empty completion:",
      len(train_ds), len(eval_ds))

Filter:   0%|          | 0/9548 [00:00<?, ? examples/s]

Filter:   0%|          | 0/503 [00:00<?, ? examples/s]

[filter] after drop empty completion: 9548 503


In [17]:
from copy import deepcopy

# prompt + completion 결합 (답변 앞에 구분자 추가 권장)
def _build_example(ex):
    prompt = ex["prompt"].rstrip()
    completion = ex["completion"].lstrip()
    text = prompt + "\n### 답변:\n" + completion
    return {"prompt_only": prompt, "text": text}

train_proc = train_ds.map(_build_example, remove_columns=[])
eval_proc = eval_ds.map(_build_example, remove_columns=[])

# 토크나이저 패딩/길이 설정
tokenizer.pad_token = tokenizer.eos_token if tokenizer.pad_token is None else tokenizer.pad_token
tokenizer.padding_side = "right"
# 너무 짧으면 completion이 잘려 labels가 -100뿐이 될 수 있음
if getattr(tokenizer, "model_max_length", 0) < 512:
    tokenizer.model_max_length = 512  # 필요 시 1024/2048

def _tok_and_mask(ex):
    out = tokenizer(ex["text"], truncation=True, max_length=tokenizer.model_max_length)
    p_ids = tokenizer(ex["prompt_only"], truncation=True, max_length=tokenizer.model_max_length)["input_ids"]
    pl = len(p_ids)

    labels = deepcopy(out["input_ids"])
    # 프롬프트 구간은 학습 대상 아님
    for i in range(min(pl, len(labels))):
        labels[i] = -100
    # 패딩은 항상 -100
    if "attention_mask" in out:
        for i, m in enumerate(out["attention_mask"]):
            if m == 0:
                labels[i] = -100
    out["labels"] = labels
    return out

train_tokenized = train_proc.map(_tok_and_mask, remove_columns=train_proc.column_names)
eval_tokenized = eval_proc.map(_tok_and_mask, remove_columns=eval_proc.column_names)

Map:   0%|          | 0/9548 [00:00<?, ? examples/s]

Map:   0%|          | 0/503 [00:00<?, ? examples/s]

Map:   0%|          | 0/9548 [00:00<?, ? examples/s]

Map:   0%|          | 0/503 [00:00<?, ? examples/s]

In [18]:
print(train_ds)
print(train_ds[0])
print(train_ds[1])

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 9548
})
{'prompt': '당신은 금융보안 전문가입니다.\n아래 질문에 대해 적절한 선택지를 출력하세요.\n\n질문: 공공민간협력(PPP) 사업에서 정부의 역할로 적절한 것은 무엇인가?\n선택지:\n1. 자금 부족 시 민간의 복지 관련 책임까지 담당한다.\n2. 프로젝트의 모든 결정 사항을 단독으로 처리한다.\n3. 민간의 이익을 우선하며 정부 역할을 최소화한다.\n4. 공적자금을 투입하고 민간과 함께 위험을 분담한다.\n\n답변:\n', 'completion': '4, 공적자금을 투입하고 민간과 함께 위험을 분담한다.'}
{'prompt': '당신은 금융보안 전문가입니다.\n아래 주관식 질문에 대해 정확하고 간략한 설명을 작성하세요.\n\n질문: CLI 감소 시 기업의 의사결정에는 어떤 영향을 미칠까요?\n\n답변:\n', 'completion': 'CLI(Consumer Leading Indicator, 소비자 선행 지표)가 감소할 경우, 이는 기업의 의사결정에 다양한 방식으로 영향을 미칠 수 있습니다. CLI는 일반적으로 소비자 신뢰도와 소비 지출 패턴을 반영하는 지표로 여겨지며, 감소는 소비자들이 경제에 대한 불확실성을 느끼거나 지출을 줄일 가능성을 암시합니다. 다음은 CLI 감소가 기업의 의사결정에 미치는 주요 영향입니다:\n\n### 1. **판매 예측 조정**  \nCLI가 감소하면 기업은 소비자 수요 감소를'}


In [19]:
print(train_tokenized)
print(train_tokenized[0])
print(train_tokenized[1])

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9548
})
{'input_ids': [27010, 5956, 24316, 8379, 879, 54, 239, 6186, 15932, 2421, 13893, 2538, 2148, 18631, 2232, 54, 239, 239, 4806, 66, 8598, 35930, 18969, 48, 12982, 88, 49, 2772, 485, 13060, 88255, 13893, 1915, 13888, 71, 239, 11310, 346, 66, 239, 57, 54, 15898, 4541, 571, 2339, 6663, 14456, 2364, 6507, 947, 102337, 54, 239, 58, 54, 72269, 2081, 3681, 20082, 47639, 6407, 1211, 54, 239, 59, 54, 2339, 6663, 21788, 6066, 1828, 3689, 6792, 16620, 1211, 54, 239, 60, 54, 42432, 54415, 12598, 632, 11482, 471, 1466, 29482, 53657, 1211, 54, 239, 239, 5176, 66, 239, 12961, 9573, 66, 239, 60, 52, 42432, 54415, 12598, 632, 11482, 471, 1466, 29482, 53657, 1211, 54], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

## 학습

In [ ]:
# Trainer 정의

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, 
    return_tensors="pt",
    label_pad_token_id=-100
)

trainer = SFTTrainer(
    model=base_model,
    processing_class=tokenizer,   # SFTTrainer면 tokenizer 대신 이 인자일 수 있음
    train_dataset=train_tokenized,      # <-- 여기!
    eval_dataset=eval_tokenized,  # 임시/예시
    args=cfg,
    data_collator=data_collator,  # <-- 여기!
    peft_config=peft_config,
)

trainer.model.print_trainable_parameters()

Truncating train dataset:   0%|          | 0/9548 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/503 [00:00<?, ? examples/s]

trainable params: 40,370,176 || all params: 7,299,995,136 || trainable%: 0.5530


In [ ]:
# Custom Callback

from transformers import TrainerCallback
from datetime import datetime
import pytz
import math

class CustomCallback(TrainerCallback):
    tz_seoul = pytz.timezone("Asia/Seoul")

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            now = datetime.now(self.tz_seoul).strftime("%Y-%m-%d %H:%M:%S")
            print(f"[{now}] step {state.global_step}\tloss {logs['loss']:.4f}", flush=True)

trainer.add_callback(CustomCallback())

In [ ]:
print("🎯 훈련 시작...")
trainer.train()

# 어댑터 저장
adapter_dir = f"../../models/{selected_model}_qlora/adapter"
os.makedirs(adapter_dir, exist_ok=True)

trainer.model.save_pretrained(adapter_dir)
tokenizer.save_pretrained(adapter_dir)

print(f"✅ 어댑터 저장 완료: {adapter_dir}")
print(f"훈련 완료 후 GPU 메모리: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

🎯 훈련 시작...


wandb: Currently logged in as: bhc5754 (bhc5754-hyalobio) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


[2025-08-10 20:11:20] step 16	loss 1.6050
[2025-08-10 20:13:04] step 32	loss 1.0693


In [ ]:
# 훈련 완료 후 어댑터 병합 (선택사항)
# 훈련이 완료되면 이 셀을 실행하여 최종 모델을 생성하세요

def merge_adapter():
    """어댑터를 베이스 모델과 병합하여 최종 모델 생성"""
    
    adapter_dir = f"../../models/{selected_model}_qlora/adapter"
    merged_dir = f"../../models/{selected_model}_qlora/merged"
    
    print("🔄 어댑터 병합 시작...")
    
    # 메모리 정리
    import gc
    if 'model' in globals(): del model
    if 'base_model' in globals(): del base_model  
    if 'trainer' in globals(): del trainer
    torch.cuda.empty_cache()
    gc.collect()
    
    # 베이스 모델 재로딩 (풀프리시전)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        attn_implementation="eager"
    )
    
    # 어댑터 병합
    model = PeftModel.from_pretrained(model, adapter_dir)
    model = model.merge_and_unload()
    
    # 저장
    model.save_pretrained(merged_dir)
    tokenizer.save_pretrained(merged_dir)
    
    print(f"✅ 최종 모델 저장 완료: {merged_dir}")
    return merged_dir

# 사용법: merge_adapter() 함수를 호출하여 실행

merge_adapter()